In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# Set up environment variables
project_id = 'eighth-arbor-396212'

os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
dataset = 'mimiciv'


In [2]:
LIMIT = True
hai_positive_patients_query = f"""SELECT
    d.subject_id,
    d.hadm_id,
    i.stay_id,
    d.icd_code,
    dicd.long_title,
    CASE
        WHEN d.icd_code LIKE 'J95851' OR d.icd_code LIKE '99731%' THEN 'VAP'
        WHEN d.icd_code LIKE '99931' OR d.icd_code LIKE '99932' OR d.icd_code LIKE '99933' OR d.icd_code LIKE 'T8021%' THEN 'CLABSI'
        WHEN d.icd_code LIKE '99664' OR d.icd_code LIKE 'T8351%' THEN 'CAUTI'
        ELSE 'Unknown'
    END AS HAI_Type
FROM
    `physionet-data.mimiciv_hosp.diagnoses_icd` d
LEFT OUTER JOIN
    `physionet-data.mimiciv_icu.icustays` i ON d.hadm_id = i.hadm_id
LEFT OUTER JOIN
    `physionet-data.mimiciv_hosp.d_icd_diagnoses` dicd ON d.icd_code = dicd.icd_code
WHERE
    d.icd_code LIKE 'J95851' OR
    d.icd_code LIKE '99731%' OR
    d.icd_code LIKE '99931' OR
    d.icd_code LIKE '99932' OR
    d.icd_code LIKE '99933' OR
    d.icd_code LIKE 'T8021%' OR
    d.icd_code LIKE '99664' OR
    d.icd_code LIKE 'T8351%'
{"LIMIT 1000" if LIMIT else ""}
"""

hai_positive_patients = run_query(hai_positive_patients_query)
hai_positive_patients

,subject_id,hadm_id,stay_id,icd_code,long_title,HAI_Type
0,10446794,27028960,<NA>,99932,Bloodstream infection due to central venous ca...,CLABSI
1,10723086,20456851,38690919,99932,Bloodstream infection due to central venous ca...,CLABSI
2,10723086,20456851,39130941,99932,Bloodstream infection due to central venous ca...,CLABSI
3,10947245,24307798,31525852,99931,Other and unspecified infection due to central...,CLABSI
4,10947245,24307798,33145790,99931,Other and unspecified infection due to central...,CLABSI
...,...,...,...,...,...,...
995,19557552,28053844,<NA>,99932,Bloodstream infection due to central venous ca...,CLABSI
996,19561278,24831073,<NA>,99931,Other and unspecified infection due to central...,CLABSI
997,19563570,22135288,<NA>,99664,Infection and inflammatory reaction due to ind...,CAUTI
998,19563570,27325833,<NA>,99664,Infection and inflammatory reaction due to ind...,CAUTI


In [3]:
def get_patients_by_long_title(titles, limit=LIMIT):
    title_conditions = " OR ".join([f"LOWER(pseq.long_title) LIKE '%{title.lower()}%'" for title in titles])

    base_query = f"""
    SELECT
        p.subject_id,
        p.hadm_id,
        i.stay_id,
        p.icd_code,
        pseq.long_title
    FROM
        `physionet-data.mimiciv_hosp.procedures_icd` p
    LEFT OUTER JOIN
        `physionet-data.mimiciv_icu.icustays` i ON p.hadm_id = i.hadm_id
    LEFT OUTER JOIN
        `physionet-data.mimiciv_hosp.d_icd_procedures` pseq ON p.icd_code = pseq.icd_code
    WHERE
        {title_conditions}
    {"LIMIT 1000" if limit else ""}
    """

    return base_query

urinary_catheter_titles = ['urinary catheter', 'ureteral catheter']
urinary_catheter_patients_query = get_patients_by_long_title(urinary_catheter_titles)
urinary_catheter_patients = run_query(urinary_catheter_patients_query)
urinary_catheter_patients

,subject_id,hadm_id,stay_id,icd_code,long_title
0,10017764,23125577,<NA>,598,Ureteral catheterization
1,10028159,20401884,<NA>,598,Ureteral catheterization
2,10036086,27288283,<NA>,598,Ureteral catheterization
3,10080443,24427299,<NA>,598,Ureteral catheterization
4,10083097,23498090,<NA>,598,Ureteral catheterization
...,...,...,...,...,...
995,19841435,26506705,<NA>,5794,Insertion of indwelling urinary catheter
996,19898796,23672782,<NA>,5794,Insertion of indwelling urinary catheter
997,19902631,28513378,<NA>,5794,Insertion of indwelling urinary catheter
998,10038852,25549907,<NA>,5795,Replacement of indwelling urinary catheter


In [4]:
venous_catheter_titles = ['central venous', 'venous catheter', 'arterial catheter', 'cardiac catheter']
venous_catheter_query = get_patients_by_long_title(venous_catheter_titles)
venous_catheter_patients = run_query(venous_catheter_query)
venous_catheter_patients

,subject_id,hadm_id,stay_id,icd_code,long_title
0,10013569,27993048,38857852,3721,Right heart cardiac catheterization
1,10013569,27993048,39673498,3721,Right heart cardiac catheterization
2,10037861,24256866,<NA>,3721,Right heart cardiac catheterization
3,10055361,22555445,37557681,3721,Right heart cardiac catheterization
4,10055361,22555445,38486782,3721,Right heart cardiac catheterization
...,...,...,...,...,...
995,12415890,28977387,<NA>,3722,Left heart cardiac catheterization
996,12416042,26092922,<NA>,3722,Left heart cardiac catheterization
997,12416773,27861680,<NA>,3722,Left heart cardiac catheterization
998,12419182,24103580,<NA>,3722,Left heart cardiac catheterization


In [5]:
ventilator_titles = ['ventilation']
ventilator_query = get_patients_by_long_title(ventilator_titles)
ventilator_patients = run_query(ventilator_query)
ventilator_patients

,subject_id,hadm_id,stay_id,icd_code,long_title
0,10002428,20321825,34807493,9390,Non-invasive mechanical ventilation
1,10089244,29469323,33563887,9390,Non-invasive mechanical ventilation
2,10104229,28690308,34675728,9390,Non-invasive mechanical ventilation
3,10146782,27318446,32413224,9390,Non-invasive mechanical ventilation
4,10163041,22909058,36574609,9390,Non-invasive mechanical ventilation
...,...,...,...,...,...
995,11959467,23536194,36974556,9671,Continuous invasive mechanical ventilation for...
996,11960875,22165732,37031815,9671,Continuous invasive mechanical ventilation for...
997,11964706,28670659,36359710,9671,Continuous invasive mechanical ventilation for...
998,11968677,26843103,31461238,9671,Continuous invasive mechanical ventilation for...
